In [1]:
import sys
!{sys.executable} -m pip install kfp >/dev/null

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import os
import kfp

In [3]:
dkube_training_op           = kfp.components.load_component_from_url("https://raw.githubusercontent.com/deepio-oc/dkube-examples/master/components/training/component.yaml")
dkube_serving_op            = kfp.components.load_component_from_url("https://raw.githubusercontent.com/deepio-oc/dkube-examples/master/components/serving/component.yaml")

In [4]:
@kfp.dsl.pipeline(
    name='dkube-mnist-pl',
    description='sample mnist pipeline with dkube components'
)
def mnist_pipeline(token):

    train       = dkube_training_op(token, '{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}',
                                    framework="tensorflow", version="2.0.0",
                                    program="dkube-examples", run_script="python mnist/train.py",
                                    datasets='["mnist"]', outputs='["mnist"]',
                                    input_dataset_mounts='["/mnist"]',
                                    output_mounts='["/opt/dkube/output"]',
                                    envs='[{"EPOCHS": "10"}]')

    serving     = dkube_serving_op(token, train.outputs['artifact'], device='cpu', 
                                   serving_image='{"image":"ocdr/tensorflowserver:2.0.0"}').after(train)

In [5]:
token  = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=token)
client.create_run_from_pipeline_func(mnist_pipeline, arguments={"token":token})

RunPipelineResult(run_id=d46276df-c3ae-47f2-b0b8-d8f898b0f890)

In [ ]:
#generate & upload pipeline (Optional)
import kfp.compiler as compiler
compiler.Compiler().compile(mnist_pipeline, "mnist-pipeline.zip")
client.upload_pipeline("mnist-pipeline.zip")